About the Dataset:

1. id: unique id for a news article
2. title: the title of a news article
3. author: author of the news article
4. text: the text of the article; could be incomplete
5. label: a label that marks whether the news article is real or fake:
           1: Fake news
           0: real News





Importing the dependencies

In [91]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import re
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer


from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

Data Loading & Preprocessing

In [92]:
df_tr = pd.read_csv('/content/drive/MyDrive/ML/siddhardhan/ml projects/fake news prediction basic/train.csv')
df_tr.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [93]:
df_tr.shape

(20800, 5)

In [94]:
# count missing values in data
df_tr.isnull().sum()

id           0
title      558
author    1957
text        39
label        0
dtype: int64

In [95]:
# replace null values with empty string
df_tr= df_tr.fillna('')

In [96]:
# merging title & author name
df_tr['content'] = df_tr['author']+' '+df_tr['title']

In [97]:
df_tr.head()

,id,title,author,text,label,content
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1,Darrell Lucus House Dem Aide: We Didn’t Even S...
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0,"Daniel J. Flynn FLYNN: Hillary Clinton, Big Wo..."
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1,Consortiumnews.com Why the Truth Might Get You...
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1,Jessica Purkiss 15 Civilians Killed In Single ...
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1,Howard Portnoy Iranian woman jailed for fictio...


Separating the features & target

In [98]:
X = df_tr['content']
X.head()

0    Darrell Lucus House Dem Aide: We Didn’t Even S...
1    Daniel J. Flynn FLYNN: Hillary Clinton, Big Wo...
2    Consortiumnews.com Why the Truth Might Get You...
3    Jessica Purkiss 15 Civilians Killed In Single ...
4    Howard Portnoy Iranian woman jailed for fictio...
Name: content, dtype: object

In [99]:
Y = df_tr['label']
Y.head()

0    1
1    0
2    1
3    1
4    1
Name: label, dtype: int64

A. Data Preprocessing

1. Lower case
2. Tokenization
3. Removing special characters
4. Removing stop words & punctuations
5. Stemming

In [100]:
# for tokenzing text into words
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [101]:
import nltk
nltk.download('stopwords')

# english stopwords
print(stopwords.words('english'))

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [102]:
# for punctuation
import string
string.punctuation

'!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'

In [103]:
# stemming
from nltk.stem.porter import PorterStemmer
ps= PorterStemmer()

In [104]:
### Function 1 for pre-processing  ###

def transform_text(text):
  # lower case
  text= text.lower()

  # tokenize
  text= nltk.word_tokenize(text)

  # remove special chars (only keep alphab. & numbers)
  l=[]
  for i in text:
    if i.isalnum():
      l.append(i)

  # remove stopwords & punctuations
  m=[]
  for i in l:
    if i not in stopwords.words('english') and i not in string.punctuation :
      m.append(i)

  # steeming the words
  n=[]
  for i in m:
    n.append(ps.stem(i))

  return ' '.join(n)

In [105]:
X.apply(transform_text)

0        darrel lucu hous dem aid even see comey letter...
1        daniel flynn flynn hillari clinton big woman c...
2                                     truth might get fire
3        jessica purkiss 15 civilian kill singl us airs...
4        howard portnoy iranian woman jail fiction unpu...
                               ...                        
20795    jerom hudson rapper trump poster child white s...
20796    benjamin hoffman playoff schedul matchup odd n...
20797    michael de la merc rachel abram maci said rece...
20798    alex ansari nato russia hold parallel exercis ...
20799                              david swanson keep aliv
Name: content, Length: 20800, dtype: object

In [106]:
X.head(8)

0    Darrell Lucus House Dem Aide: We Didn’t Even S...
1    Daniel J. Flynn FLYNN: Hillary Clinton, Big Wo...
2    Consortiumnews.com Why the Truth Might Get You...
3    Jessica Purkiss 15 Civilians Killed In Single ...
4    Howard Portnoy Iranian woman jailed for fictio...
5    Daniel Nussbaum Jackie Mason: Hollywood Would ...
6     Life: Life Of Luxury: Elton John’s 6 Favorite...
7    Alissa J. Rubin Benoît Hamon Wins French Socia...
Name: content, dtype: object

In [107]:
### Function 2 for pre-processing  ###

def transf_text_2(content):
  # check for only alphabets
  transfm = re.sub('[^a-zA-Z]',' ',content)

  # turn to lower case
  transfm = transfm.lower()

  # tokenize
  transfm = transfm.split()

  # remove stopwords & then stem it
  transfm = [ps.stem(word) for word in transfm if word not in stopwords.words('english')]

  return " ".join(transfm)

In [108]:
X= X.apply(transf_text_2)
X

0        darrel lucu hous dem aid even see comey letter...
1        daniel j flynn flynn hillari clinton big woman...
2                   consortiumnew com truth might get fire
3        jessica purkiss civilian kill singl us airstri...
4        howard portnoy iranian woman jail fiction unpu...
                               ...                        
20795    jerom hudson rapper trump poster child white s...
20796    benjamin hoffman n f l playoff schedul matchup...
20797    michael j de la merc rachel abram maci said re...
20798    alex ansari nato russia hold parallel exercis ...
20799                            david swanson keep f aliv
Name: content, Length: 20800, dtype: object

In [109]:
X.values

array(['darrel lucu hous dem aid even see comey letter jason chaffetz tweet',
       'daniel j flynn flynn hillari clinton big woman campu breitbart',
       'consortiumnew com truth might get fire', ...,
       'michael j de la merc rachel abram maci said receiv takeov approach hudson bay new york time',
       'alex ansari nato russia hold parallel exercis balkan',
       'david swanson keep f aliv'], dtype=object)

In [110]:
# converting textual data to numerical data

vectorizer = TfidfVectorizer()

In [111]:
X_transf= vectorizer.fit_transform(X)
X_transf.toarray()

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

Splitting the data to training & test data

In [112]:
X_train, X_test, Y_train, Y_test = train_test_split(X_transf,Y, test_size=0.2,stratify=Y,random_state=2)

Training the model: Logistic Regression

In [113]:
model = LogisticRegression()

In [114]:
model.fit(X_train,Y_train)

LogisticRegression()

Prediction & Evaluation of model

In [115]:
Y_pred= model.predict(X_test)

In [116]:
accuracy_score(Y_test,Y_pred)

0.9790865384615385

Build a predictive system

In [130]:
sample_news = ''' Texas Oil Fields Rebound From Price Lull, but Jobs Are Left Behind - The New York Times
'''

# print(sample_news)

transfm_sample_news = [transf_text_2(sample_news)]
# print(transfm_sample_news)

vectorized_news= vectorizer.transform(transfm_sample_news)

prediction = model.predict(vectorized_news)

print(prediction)

if prediction[0]== 1:
  print('Fake News')
else:
  print('Real News')


[0]
Real News
